# MTAOS addAberration

Notebooks to show how the addAberration command in MTAOS works.

(using OFC code before Tiago's rewrite in April 2021)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from lsst.ts.ofc.ctrlIntf.OFCCalculationFactory import OFCCalculationFactory
from lsst.ts.wep.Utility import CamType
from lsst.ts.ofc.Utility import getConfigDir, DofGroup, InstName
from lsst.ts.ofc.ctrlIntf.FWHMSensorData import FWHMSensorData
from lsst.ts.wep.ctrlIntf.MapSensorNameAndId import MapSensorNameAndId
from lsst.ts.wep.ctrlIntf.SensorWavefrontError import SensorWavefrontError
from lsst.ts.wep.Utility import FilterType
from lsst.ts.wep.bsc.CamFactory import CamFactory
from lsst.ts.MTAOS.CollOfListOfWfErr import CollOfListOfWfErr

In [ ]:
getConfigDir()

In [ ]:
#watch out, do not use from lsst.ts.wep.Utility.CamType.ComCam
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)

In [ ]:
ofc.ztaac.dataShare.getDofIdx()

In [ ]:
comcam = CamFactory.createCam(CamType.ComCam)

In [ ]:
detectorList = comcam.getWfsCcdList()

In [ ]:
sensorIdList = ofc.ztaac.mapSensorNameToId(detectorList)

In [ ]:
intrinsic_zk = ofc.ztaac.dataShare.getIntrinsicZk(FilterType.REF, ofc.ztaac.dataShare.getFieldIdx(detectorList))

## If you pass Intrinsic aberrations to OFC, you get all corrections = 0

In [ ]:
listOfWfErr = []
for sensorId, annularZernikePoly in zip(sensorIdList, intrinsic_zk):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr.append(sensorWavefrontError)

In [ ]:
(
    m2HexapodCorrection,
    cameraHexapodCorrection,
    m1m3Correction,
    m2Correction,
) = ofc.calculateCorrections(listOfWfErr)

In [ ]:
print(m2HexapodCorrection.getCorrection())
print(cameraHexapodCorrection.getCorrection())
print(m1m3Correction.getZForces())
print(m2Correction.getZForces())

### Add 1um z7 (We tell OFC that we measure intrinsic - (1um z7), so that it adds 1um z7 to go back to intrinsic

In [ ]:
add_coma = np.array(intrinsic_zk, copy=True)
add_coma[:, 7-4] -= 1 #z4 has index 0

listOfWfErr_coma = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_coma.append(sensorWavefrontError)
    
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)

(
    m2HexapodCorrection_cx1,
    cameraHexapodCorrection_cx1,
    m1m3Correction_cx1,
    m2Correction_cx1,
) = ofc.calculateCorrections(listOfWfErr_coma)

In [ ]:
print(m2HexapodCorrection_cx1.getCorrection())
print(cameraHexapodCorrection_cx1.getCorrection())
print(m1m3Correction_cx1.getZForces())
print(m2Correction_cx1.getZForces())

In [ ]:
(
    m2HexapodCorrection_cx2,
    cameraHexapodCorrection_cx2,
    m1m3Correction_cx2,
    m2Correction_cx2,
) = ofc.calculateCorrections(listOfWfErr_coma)

print(np.array(m2HexapodCorrection_cx2.getCorrection())[:5]/np.array(m2HexapodCorrection_cx1.getCorrection())[:5])

### This is because the default control strategy is "x00"

In [ ]:
ofc.ztaac.dataShare.xRef

In [ ]:
dofIdx = ofc.ztaac.dataShare.getDofIdx()
print(ofc.ztaac.optCtrl.getState0(dofIdx))
print(ofc.ztaac.optCtrl.getState(dofIdx))

In [ ]:
add_coma = np.array(intrinsic_zk, copy=True)
add_coma[:, 7-4] -= 1 #z4 has index 0

listOfWfErr_coma = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_coma.append(sensorWavefrontError)
    
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)

(
    m2HexapodCorrection_cx1,
    cameraHexapodCorrection_cx1,
    m1m3Correction_cx1,
    m2Correction_cx1,
) = ofc.calculateCorrections(listOfWfErr_coma)

state0 = np.zeros(ofc.ztaac.optCtrl.getNumOfState0())
ofc.ztaac.optCtrl.setState(state0)

(
    m2HexapodCorrection_cx2,
    cameraHexapodCorrection_cx2,
    m1m3Correction_cx2,
    m2Correction_cx2,
) = ofc.calculateCorrections(listOfWfErr_coma)

print(np.array(m2HexapodCorrection_cx2.getCorrection())[:5]/np.array(m2HexapodCorrection_cx1.getCorrection())[:5])

### What happens if we set control strategy to "x0"

In [ ]:
ofc.ztaac.dataShare.xRef 

In [ ]:
add_coma = np.array(intrinsic_zk, copy=True)
add_coma[:, 7-4] -= 1 #z4 has index 0

listOfWfErr_coma = []
for sensorId, annularZernikePoly in zip(sensorIdList, add_coma):
    #print(sensorId)
    sensorWavefrontError = SensorWavefrontError(numOfZk=19)
    sensorWavefrontError.setSensorId(sensorId)
    sensorWavefrontError.setAnnularZernikePoly(annularZernikePoly)
    listOfWfErr_coma.append(sensorWavefrontError)
    
ofc = OFCCalculationFactory.getCalculator(InstName.COMCAM, None)
ofc.ztaac.dataShare.xRef = 'x0'

(
    m2HexapodCorrection_cx1,
    cameraHexapodCorrection_cx1,
    m1m3Correction_cx1,
    m2Correction_cx1,
) = ofc.calculateCorrections(listOfWfErr_coma)

#state0 = np.zeros(ofc.ztaac.optCtrl.getNumOfState0())
#ofc.ztaac.optCtrl.setState(state0)

(
    m2HexapodCorrection_cx2,
    cameraHexapodCorrection_cx2,
    m1m3Correction_cx2,
    m2Correction_cx2,
) = ofc.calculateCorrections(listOfWfErr_coma)

print(np.array(m2HexapodCorrection_cx2.getCorrection())[:5]/np.array(m2HexapodCorrection_cx1.getCorrection())[:5])

In [ ]:
(
    m2HexapodCorrection,
    cameraHexapodCorrection,
    m1m3Correction,
    m2Correction,
) = ofc.resetOfcState()

In [ ]:
print(m2HexapodCorrection.getCorrection())
print(cameraHexapodCorrection.getCorrection())
print(m1m3Correction.getZForces())
print(m2Correction.getZForces())